In [18]:
import torch
from bilstm_crf import BiLSTMCRF

# Load data

In [ ]:
import json

# load vocab
with open('data/vocab.txt', 'r') as f:
    vocab = f.read().split('\n')
len(vocab)

# load tag_to_id
with open('data/tag_to_id.json', 'r') as f:
    tag_to_id = json.load((f))

In [12]:
# load train and dev data
TRAIN_PATH = 'data/process_data/train.json'
DEV_PATH = 'data/process_data/dev.json'

with open(TRAIN_PATH, 'r') as f:
    train_data = json.load(f)

with open(DEV_PATH, 'r') as f:
    dev_data = json.load(f)

In [ ]:
train_sentences = train_data['text'].values()
dev_sentences = dev_data['text'].values()

train_labels = train_data['labels'].values()
dev_labels = dev_data['labels'].values()

In [1]:
# Tokenize data
def tokenize(data):
    tokenized_data = []
    for sentence in data:
        tokens = []
        for word in sentence.split():
            tokens.append(word)
        tokenized_data.append(tokens)
    return tokenized_data

# Convert data to ids
def convert_to_ids(data, vocab, max_len=256):
    id_data = []

    pad_token_id = vocab.index('<PAD>')
    ukn_token_id = vocab.index('<UKN>')
    for sentence in data:
        ids = []
        for word in sentence.split():
            if word in vocab:
                ids.append(vocab.index(word))
            else:
                ids.append(ukn_token_id)

        if len(ids) < max_len:
            ids += [pad_token_id] * (max_len - len(ids))
        id_data.append(ids)
        
    return id_data

In [ ]:
train_tokenized = convert_to_ids(train_data, vocab, tag_to_id)
dev_tokenized = convert_to_ids(dev_data, vocab, tag_to_id)

# Model

In [20]:
# load fasttext embedding
import numpy as np
from gensim.models import FastText

weights_path = 'pretrained-weights/cc.vi.300.vec'

ft = FastText.load_fasttext_format(weights_path)

In [7]:
# load model
span_detect_model = BiLSTMCRF(len(vocab), tag_to_id)

In [8]:
span_detect_model.eval()

BiLSTMCRF(
  (word_embeds): Embedding(15854, 384)
  (lstm): LSTM(384, 192, bidirectional=True)
  (hidden2tag): Linear(in_features=384, out_features=10, bias=True)
  (crf): CRF(num_tags=10)
)

In [17]:
example = train_input_ids[0]
example.shape

torch.Size([1, 384])

In [14]:
# run model on example
with torch.no_grad():
    output = span_detect_model(example)

output[0].shape

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Hii\miniconda3\envs\udemy\Lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Hii\AppData\Local\Temp\ipykernel_12176\2773444422.py", line 3, in <module>
    output = span_detect_model(example)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Hii\miniconda3\envs\udemy\Lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\CLOUDX\Courses\nlp\aspect-based-sentiment-analysis\bilstm_crf.py", line 61, in forward
  File "d:\CLOUDX\Courses\nlp\aspect-based-sentiment-analysis\bilstm_crf.py", line 33, in _get_lstm_features
    raise ValueError('units must be one of lstm, gru or rnn')
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Hii\miniconda3\envs\udemy\Lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
   